# This code performs classification on the forest cover type dataset using a deep learning model.


The code begins by importing the necessary libraries for data processing and model building.

In [2]:
import pandas as pd
from collections import Counter
from sklearn.preprocessing import LabelEncoder
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  InputLayer
from tensorflow.keras.layers import  Dense
from sklearn.metrics import classification_report
import numpy as np

In [3]:
# We will use pandas to read the dataset
dataset = pd.read_csv("cover_data.csv")

# Let's check the shape of the dataset and other info
print(dataset.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581012 entries, 0 to 581011
Data columns (total 55 columns):
 #   Column                              Non-Null Count   Dtype
---  ------                              --------------   -----
 0   Elevation                           581012 non-null  int64
 1   Aspect                              581012 non-null  int64
 2   Slope                               581012 non-null  int64
 3   Horizontal_Distance_To_Hydrology    581012 non-null  int64
 4   Vertical_Distance_To_Hydrology      581012 non-null  int64
 5   Horizontal_Distance_To_Roadways     581012 non-null  int64
 6   Hillshade_9am                       581012 non-null  int64
 7   Hillshade_Noon                      581012 non-null  int64
 8   Hillshade_3pm                       581012 non-null  int64
 9   Horizontal_Distance_To_Fire_Points  581012 non-null  int64
 10  Wilderness_Area1                    581012 non-null  int64
 11  Wilderness_Area2                    581012 non-null 

Feature Extraction

In [6]:
# Features (X)
features = dataset.iloc[:, 0:-1]

# Labels (y)
labels = dataset.iloc[:, -1]

print(Counter(labels))

Counter({2: 283301, 1: 211840, 3: 35754, 7: 20510, 6: 17367, 5: 9493, 4: 2747})


Next, the dataset is split into training and testing sets using train_test_split() from sklearn. The test_size parameter is set to 0.3, indicating a 70:30 train-test split.

To prepare the data for modeling, the numerical features are standardized using StandardScaler() from sklearn. The columns to be standardized are selected based on their data types (float64 and int64) using select_dtypes(). The ColumnTransformer is used to apply standardization only to the numerical columns, while leaving the remaining columns untouched.

The training and testing features are then transformed using the ColumnTransformer object, applying the standardization to the numerical features.

The labels are encoded using LabelEncoder() to convert them into integer labels. This step is necessary for compatibility with the model.

The integer labels are further transformed into binary vectors using to_categorical() from tensorflow.keras.utils. This step converts the labels into a one-hot encoded format, suitable for multi-class classification.


In [31]:
from sklearn.model_selection import train_test_split

features_train, features_test, labels_train, labels_test = train_test_split(features, labels, test_size = 0.3, random_state=42)

# Standardize
numerical_features = features.select_dtypes(include=['float64', 'int64'])

numerical_columns = numerical_features.columns

# Create a ct object
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

ct = ColumnTransformer([("only numeric", StandardScaler(), numerical_columns)], remainder='passthrough')

# Transform the test data features_test using ct
features_train_scaled = ct.fit_transform(features_train)
features_test_scaled = ct.transform(features_test)

# Convert the labels into integers
label_encoder = LabelEncoder()
labels_train_encoded = label_encoder.fit_transform(labels_train)
labels_test_encoded = label_encoder.transform(labels_test)

# Convert the integer labels into binary vectors
labels_train = tensorflow.keras.utils.to_categorical(labels_train_encoded)
labels_test = tensorflow.keras.utils.to_categorical(labels_test_encoded)

The model is initialized as a sequential model using Sequential() from tensorflow.keras.models. This type of model allows stacking multiple layers sequentially.

The input layer is added to the model using InputLayer() with the shape parameter set to the number of features in the training data.

A hidden layer with 32 units and ReLU activation is added to the model using Dense(). This layer introduces non-linearity to the model.

Another hidden layer with 16 units and ReLU activation is added to the model, further capturing complex patterns in the data.

The output layer with 7 neurons (corresponding to the 7 classes) and softmax activation is added to the model. Softmax activation ensures that the output values represent class probabilities.

The model summary is printed using model.summary(), providing a concise overview of the model architecture, the number of parameters, and the shape of the output at each layer.

In [34]:
model = Sequential()

# Input layer
input = InputLayer(input_shape = (features_train.shape[1], ))
model.add(input)

# Hidden layer
model.add(Dense(32, activation="relu"))
# Add another hidden layer
model.add(Dense(16, activation="relu"))

# Output layer with 7 neurons since we have 7 classes
model.add(Dense(7, activation="softmax"))

# Print the summary of the model
print(model.summary())

Model: "sequential_8"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_16 (Dense)            (None, 32)                1760      
                                                                 
 dense_17 (Dense)            (None, 16)                528       
                                                                 
 dense_18 (Dense)            (None, 7)                 119       
                                                                 
Total params: 2,407
Trainable params: 2,407
Non-trainable params: 0
_________________________________________________________________
None


The model is compiled using compile() with the categorical cross-entropy loss, accuracy metric, and Adam optimizer. The learning rate for the optimizer is set to 0.001.

An early stopping callback is defined using EarlyStopping(). It monitors the loss during training and stops training if no improvement is observed after 3 epochs.

The model is trained using fit(), specifying the training features, labels, number of epochs, batch size, verbosity, and the early stopping callback. The training process is displayed with progress information.

In [35]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

opt = Adam(learning_rate = 0.001)

model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer = opt)

# Define the early stopping callback
early_stopping = EarlyStopping(monitor='loss', patience=3)

# Fit the model with early stopping
model.fit(features_train_scaled, labels_train, epochs=20, batch_size=32, verbose=1, callbacks=[early_stopping])

Epoch 1/20
12710/12710 [==============================] - 16s 1ms/step - loss: 0.5953 - accuracy: 0.7467
Epoch 2/20
12710/12710 [==============================] - 18s 1ms/step - loss: 0.5108 - accuracy: 0.7804
Epoch 3/20
12710/12710 [==============================] - 15s 1ms/step - loss: 0.4856 - accuracy: 0.7920
Epoch 4/20
12710/12710 [==============================] - 15s 1ms/step - loss: 0.4692 - accuracy: 0.8000
Epoch 5/20
12710/12710 [==============================] - 15s 1ms/step - loss: 0.4596 - accuracy: 0.8045
Epoch 6/20
12710/12710 [==============================] - 15s 1ms/step - loss: 0.4518 - accuracy: 0.8076
Epoch 7/20
12710/12710 [==============================] - 16s 1ms/step - loss: 0.4459 - accuracy: 0.8106
Epoch 8/20
12710/12710 [==============================] - 16s 1ms/step - loss: 0.4415 - accuracy: 0.8131
Epoch 9/20
12710/12710 [==============================] - 16s 1ms/step - loss: 0.4377 - accuracy: 0.8144
Epoch 10/20
12710/12710 [==============================

The model predicts the labels for the testing features using model.predict(), producing predicted class probabilities.

The predicted labels and true labels are obtained by finding the index of the maximum probability using np.argmax(). This converts the one-hot encoded vectors back into class labels.

Finally, the classification report is printed using classification_report() from sklearn.metrics. This report provides precision, recall, F1-score, and support for each class, allowing for detailed evaluation of the model's performance.

In [36]:
y_pred = model.predict(features_test_scaled)

y_pred_labels = np.argmax(y_pred, axis=1)

y_true_labels = np.argmax(labels_test, axis=1)

print(classification_report(y_true_labels, y_pred_labels))


              precision    recall  f1-score   support

           0       0.85      0.78      0.81     63556
           1       0.82      0.90      0.86     85078
           2       0.77      0.85      0.81     10638
           3       0.82      0.54      0.65       795
           4       0.64      0.39      0.48      2941
           5       0.62      0.51      0.56      5227
           6       0.92      0.70      0.79      6069

    accuracy                           0.82    174304
   macro avg       0.78      0.67      0.71    174304
weighted avg       0.82      0.82      0.82    174304

